In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')
store = pd.read_csv('store.csv')

df_new = df.merge(store, on = ['Store'], how = 'inner')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import numpy as np
df_new['Date'] = pd.to_datetime(df_new['Date'], infer_datetime_format=True)
df_new["Month"] = df_new["Date"].dt.month
df_new["Quarter"] = df_new["Date"].dt.quarter
df_new["Year"] = df_new["Date"].dt.year
df_new["Day"] = df_new["Date"].dt.day
df_new["Week"] = df_new["Date"].dt.week
df_new["Season"] = np.where(df_new["Month"].isin([3,4,5]),"Spring", np.where(df_new["Month"].isin([6,7,8]),
                      "Summer",
                           np.where(df_new["Month"].isin
                           ([9,10,11]),"Fall",
                                np.where(df_new["Month"].isin
                                ([12,1,2]),"Winter","None"))))

In [4]:
df_new['CompetitionDistance'].fillna(df_new['CompetitionDistance'].mode()[0])

import seaborn as sns
sns.set(style="whitegrid")
ax = sns.barplot(x="Season", y="Sales", data=df_new)

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def create_ohe(df, col):
    le = LabelEncoder()
    a=le.fit_transform(df_new[col]).reshape(-1,1)
    ohe = OneHotEncoder(sparse=False)
    column_names = [col+ "_"+ str(i) for i in le.classes_]
    return(pd.DataFrame(ohe.fit_transform(a),columns =column_names))

In [6]:
numeric_columns = ["Customers","Open","Promo","Promo2",
"StateHoliday","SchoolHoliday","CompetitionDistance"]
categorical_columns = ["DayOfWeek","Quarter","Month","Year",
"StoreType","Assortment","Season"]


In [7]:
print(numeric_columns)

['Customers', 'Open', 'Promo', 'Promo2', 'StateHoliday', 'SchoolHoliday', 'CompetitionDistance']


In [8]:
temp = df_new[numeric_columns]
for column in categorical_columns:
    temp_df = create_ohe(df_new,column)
    temp = pd.concat([temp,temp_df],axis=1)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [ ]:
print(temp.columns[temp.dtypes=="object"])

Index(['StateHoliday'], dtype='object')


In [ ]:
temp["StateHoliday"]= np.where(temp["StateHoliday"]== '0',0,1)
temp.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [ ]:
temp[['CompetitionDistance']] = temp[['CompetitionDistance']].fillna(23)
temp.isnull().any()

Customers              False
Open                   False
Promo                  False
Promo2                 False
StateHoliday           False
SchoolHoliday          False
CompetitionDistance    False
DayOfWeek_1            False
DayOfWeek_2            False
DayOfWeek_3            False
DayOfWeek_4            False
DayOfWeek_5            False
DayOfWeek_6            False
DayOfWeek_7            False
Quarter_1              False
Quarter_2              False
Quarter_3              False
Quarter_4              False
Month_1                False
Month_2                False
Month_3                False
Month_4                False
Month_5                False
Month_6                False
Month_7                False
Month_8                False
Month_9                False
Month_10               False
Month_11               False
Month_12               False
Year_2013              False
Year_2014              False
Year_2015              False
StoreType_a            False
StoreType_b   

In [ ]:
from sklearn.model_selection import train_test_split
target = df_new[["Sales"]]
x_train, x_test, y_train, y_test = train_test_split(temp,
target,test_size=0.2,random_state=2018)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train,
y_train,test_size=0.1,random_state=2018)

In [ ]:
print("Shape of x_train:",x_train.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of x_test:",x_test.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_val:",y_val.shape)
print("Shape of y_test:",y_test.shape)

Shape of x_train: (732390, 44)
Shape of x_val: (81377, 44)
Shape of x_test: (203442, 44)
Shape of y_train: (732390, 1)
Shape of y_val: (81377, 1)
Shape of y_test: (203442, 1)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(150,input_dim = 44,activation="relu"))
model.add(Dense(1,activation = "linear"))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
x_train.head(5)

,Customers,Open,Promo,Promo2,StateHoliday,SchoolHoliday,CompetitionDistance,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,...,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,Season_Fall,Season_Spring,Season_Summer,Season_Winter
157696,741,1,1,0,0,0,4130.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
517215,785,1,1,1,0,0,1010.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
799211,1080,1,1,0,1,0,29190.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
592833,1175,1,1,0,0,0,200.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
768559,524,1,0,1,0,1,2030.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
model.compile(optimizer='rmsprop',loss="mean_absolute_error",
metrics=["mean_absolute_error"])

In [ ]:
print("MAE for Test Data:",abs(y_test - y_train.mean()).mean())

MAE for Test Data: Sales    2883.587604
dtype: float64


In [ ]:
model.fit(x_train.values,y_train.values, validation_data=
(x_val,y_val),epochs=10,batch_size=64)

Instructions for updating:
Use tf.cast instead.
Train on 732390 samples, validate on 81377 samples
Epoch 1/10
732390/732390 [==============================] - 17s 23us/step - loss: 972.9450 - mean_absolute_error: 972.9450 - val_loss: 864.4434 - val_mean_absolute_error: 864.4434
Epoch 2/10
732390/732390 [==============================] - 14s 19us/step - loss: 809.6086 - mean_absolute_error: 809.6086 - val_loss: 774.1692 - val_mean_absolute_error: 774.1692
Epoch 3/10
732390/732390 [==============================] - 13s 18us/step - loss: 769.1626 - mean_absolute_error: 769.1626 - val_loss: 868.6705 - val_mean_absolute_error: 868.6705
Epoch 4/10
732390/732390 [==============================] - 13s 18us/step - loss: 755.1440 - mean_absolute_error: 755.1440 - val_loss: 768.9005 - val_mean_absolute_error: 768.9005
Epoch 5/10
732390/732390 [==============================] - 13s 18us/step - loss: 745.4215 - mean_absolute_error: 745.4215 - val_loss: 717.4842 - val_mean_absolute_error: 717.4842
E

In [ ]:
result = model.evaluate(x_test.values,y_test.values)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result
    [i],2)))

In [ ]:
model = Sequential()
model.add(Dense(150,input_dim = 44,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
history = model.fit(x_train,y_train, validation_data=(x_val,
y_val),epochs=10,batch_size=64)

In [ ]:
result = model.evaluate(x_test,y_test)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result
    [i],2)))

In [ ]:
model = Sequential()
model.add(Dense(150,input_dim = 44,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
model.fit(x_train,y_train, validation_data=(x_val,y_val),
epochs=15,batch_size=64)
result = model.evaluate(x_test,y_test)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result
    [i],2)))

In [ ]:
model = Sequential()
model.add(Dense(350,input_dim = 44,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
model.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=15,batch_size=64)
result = model.evaluate(x_test,y_test)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",
    str(round(result[i],2)))

In [ ]:
from keras.callbacks import History
history = History()
model = Sequential()
model.add(Dense(350,input_dim = 44,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
model.fit(x_train,y_train, validation_data=(x_val,y_val),
epochs=15,batch_size=64,callbacks=[history])
result = model.evaluate(x_test,y_test)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result
    [i],2)))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model's Training & Validation loss across epochs")
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()